# CONSTANTS

In [1]:
import math

GLOBAL = {
    'embedded_length': 512,
    'continuity_length': 64,
    'n_vocab': 29
}

# Transformer (Top level)
TRANS_CONST = {
    'n_attention_layers': 8,
    'n_attention_heads': 8,    

    'max_output_length': 6,
    
    'embedding_dic_size': GLOBAL['n_vocab'], 
    'embedded_vec_size': GLOBAL['embedded_length'],
    
    # 'pos_encoding_input': GLOBAL['embedded_length'],
    # 'pos_encoding_output': GLOBAL['embedded_length'],
    
    'linear_input': GLOBAL['embedded_length'],
    'linear_output': GLOBAL['n_vocab'] # output vocab size
}

# Encoder, EncoderLayer
ENCODER_CONST = {
    'norm1_size': GLOBAL['embedded_length'], # same as input matrix width
    'norm2_size': GLOBAL['embedded_length'],

    # maybe rename these two, it's just for knowing the input dim and the dim that the FF layer will work with
    'ff1': GLOBAL['embedded_length'], 
    'ff2': GLOBAL['embedded_length'] * 4
}

# Decoder, DecoderLayer
DECODER_CONST = {
    'norm1_size': GLOBAL['embedded_length'], # same as input matrix width
    'norm2_size': GLOBAL['embedded_length'],
    'norm3_size': GLOBAL['embedded_length'],

    'ff1': GLOBAL['embedded_length'],#TODO RENAME
    'ff2': GLOBAL['embedded_length'] * 4#TODO RENAME
}

# MultiHeadAttention, SingleHeadAttention
ATTENTION_CONST = {
    'mh_concat_width': GLOBAL['continuity_length']*TRANS_CONST['n_attention_heads'], # single head attention width * number of heads
    'mh_output_width': GLOBAL['embedded_length'], #TODO - I'm just guessing this. Didn't see in illustrated transformer. Since we have to use this for the add & norm layer though it has to be the same as the input width (I think)

    # W_q weight matrix 
    'sh_linear1_input': GLOBAL['embedded_length'], # same as embedded length to end up with n_words x 64
    'sh_linear1_output': GLOBAL['continuity_length'], # specified in the paper
    # W_k weight matrix 
    'sh_linear2_input': GLOBAL['embedded_length'], # same as embedded length to end up with n_words x 64
    'sh_linear2_output': GLOBAL['continuity_length'], # specified in the paper
    # W_v weight matrix 
    'sh_linear3_input': GLOBAL['embedded_length'], # same as embedded length to end up with n_words x 64
    'sh_linear3_output': GLOBAL['continuity_length'], # specified in the paper
    
    'sh_scale_factor': math.sqrt(GLOBAL['continuity_length']) # specified in the paper, square root of dimension of key vector/matrix (64)
}

# FeedForward
FEEDFORWARD_CONST = {
    'dropout': 0.1
}



# FEEDFORWARD

In [2]:
import torch.nn as nn

class FeedForward(nn.Module):
    def __init__(self, dim_model, dim_ff, dropout=FEEDFORWARD_CONST['dropout']):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(dim_model, dim_ff)
        # self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_ff, dim_model)

    def forward(self, inputs):
        x = inputs
        x = self.linear1(x)
        x = nn.functional.relu(x) 
        # x = self.dropout(x) 
        x = self.linear2(x) 
        return x


# MULTIHEADATTENTION

In [3]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads):
        super(MultiHeadAttention, self).__init__()

        self.linear = nn.Linear(64, 512)
        self.wQ = nn.Linear(ATTENTION_CONST['sh_linear1_input'], ATTENTION_CONST['sh_linear1_output'])
        self.wK = nn.Linear(ATTENTION_CONST['sh_linear2_input'], ATTENTION_CONST['sh_linear2_output'])
        self.wV = nn.Linear(ATTENTION_CONST['sh_linear3_input'], ATTENTION_CONST['sh_linear3_output'])

    def forward(self, q, k, v):
        q = self.wQ(q)
        k = self.wK(k)
        v = self.wV(v)

        # split heads - I think they do this instead of a loop
        x, attention_weights = self.applyHeads(q, k, v)
        # transpose ?
        # reshape ?
        x = self.linear(x)
        return x, attention_weights

    def applyHeads(self, q, k, v, mask=None):
        x = torch.matmul(q, k.permute(1, 0)) 
        # scale x
        # add mask
        attention_weights = nn.Softmax(dim=-1)(x)
        x = torch.matmul(attention_weights, v)
        return x, attention_weights



# DECODER

In [4]:
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, n_layers, n_attention_heads):
        super(Decoder, self).__init__()
        
        #self.embedding
        #self.pos_encoding
        #self.dropout
        self.decoderLayers = nn.ModuleList([DecoderLayer(n_attention_heads) for _ in range(n_layers)])

    def forward(self, inputs, encoderOut):
        x = inputs
        #x = self.embedding(x)
        #x = x * root of dim, whatever that is exactly
        #x = self.pos_encoding(x)
        #x = self.dropout(x)

        attention_weights = []
        for layer in self.decoderLayers:
            x, att1, att2 = layer(x, encoderOut) 
            attention_weights.append([att1, att2])

        return x, attention_weights

class DecoderLayer(nn.Module):
    def __init__(self, n_attention_heads):
        super(DecoderLayer, self).__init__()

        self.mhattention1 = MultiHeadAttention(n_attention_heads)
        self.mhattention2 = MultiHeadAttention(n_attention_heads)
        self.norm1 = nn.LayerNorm(DECODER_CONST['norm1_size'])
        self.norm2 = nn.LayerNorm(DECODER_CONST['norm2_size'])
        self.norm3 = nn.LayerNorm(DECODER_CONST['norm3_size'])
        self.dropout1 = nn.Dropout()
        self.dropout2 = nn.Dropout()
        self.dropout3 = nn.Dropout()
        self.feedforward = FeedForward(DECODER_CONST['ff1'], DECODER_CONST['ff2'])

    def forward(self, inputs, encoderOut):
        x = inputs
        z = x
        x, att1 = self.mhattention1(x, x, x)
        x = self.dropout1(x)
        x = z + x        
        x = self.norm1(x)
        z = x
        x, att2 = self.mhattention2(x, encoderOut, encoderOut) 
        x = self.dropout2(x)
        x = z + x
        x = self.norm2(x)
        z = x
        x = self.feedforward(x)
        x = self.dropout3(x)
        x = z + x
        x = self.norm3(x)
        return x, att1, att2



# ENCODER

In [5]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, n_layer, n_attention_heads):
        super(Encoder, self).__init__()
        
        #self.embedding
        #self.pos_encoding
        #self.dropout
        self.encoderLayers = nn.ModuleList([EncoderLayer(n_attention_heads) for _ in range(n_layer)])

    def forward(self, inputs):
        x = inputs
        #x = self.embedding(x)
        #x = x * root of dim, whatever that is exactly
        #x = self.pos_encoding(x)
        #x = self.dropout(x)

        for layer in self.encoderLayers:
            x = layer(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, n_attention_heads):
        super(EncoderLayer, self).__init__()

        self.mhattention = MultiHeadAttention(n_attention_heads)
        self.norm1 = nn.LayerNorm(ENCODER_CONST['norm1_size'])
        self.norm2 = nn.LayerNorm(ENCODER_CONST['norm2_size'])
        self.dropout1 = nn.Dropout()
        self.dropout2 = nn.Dropout()
        self.feedforward = FeedForward(ENCODER_CONST['ff1'], ENCODER_CONST['ff2'])

    def forward(self, inputs):
        x = inputs 
        z = x
        x, _ = self.mhattention(x, x, x)
        x = self.dropout1(x)
        x = z + x
        x = self.norm1(x)
        z = x
        x = self.feedforward(x)
        x = self.dropout2(x)
        x = z + x
        x = self.norm2(x) 
        return x



# TRANSFORMER

In [6]:
import torch
import torch.nn as nn
import numpy

class Transformer(nn.Module):
    def __init__(self, n_layers=TRANS_CONST['n_attention_layers'], n_attention_heads=TRANS_CONST['n_attention_heads']):
        super(Transformer, self).__init__()

        self.encoder = Encoder(n_layers, n_attention_heads)
        self.decoder = Decoder(n_layers, n_attention_heads)
        self.embedding = nn.Embedding(TRANS_CONST['embedding_dic_size'], TRANS_CONST['embedded_vec_size'])
        # self.posEncoding = #TODO
        self.linear = nn.Linear(TRANS_CONST['linear_input'], TRANS_CONST['linear_output'])
        self.softmax = nn.Softmax(dim=1)

    def __call__(self, inputs=None):
        if inputs != None: 
            raise NotImplementedError

        import random
        inputs = []
        for _ in range(13): inputs.append(numpy.zeros(26)) # 26 is vocab size, should be constant; 13 is just a random amount of words in the sequence
        inputs = torch.Tensor(inputs)
        for i in inputs: i[random.randint(0, len(i) - 1)] = 1

        return self.forward(inputs.long())

    def forward(self, inputs):
        x = self.doEmbedding(inputs)
        encoderOut = self.encoder(x)
        x, weights = self.decoder(x, encoderOut)
        x, weights = self.decoder(x, encoderOut)
        x = self.linear(x)
        x = self.softmax(x)
        return x, weights

    def doEmbedding(self, inputs):
        x = inputs.nonzero()[:, 1] # this gets all indices of nonzero values from the inputs matrix
        x = self.embedding(x)
        # x = self.posEncoding(x)
        return x








# Check Shapes

In [7]:
import random
inputs = []
for _ in range(13): inputs.append(numpy.zeros(26)) # 26 is vocab size, should be constant; 13 is just a random amount of words in the sequence
inputs = torch.Tensor(inputs)
for i in inputs: i[random.randint(0, len(i) - 1)] = 1

sample_enc = Encoder(1, 1)
sample_dec = Decoder(1, 1)
sample_tf = Transformer()

#sample_x = inputs
#print(sample_x.shape)
#sample_x = sample_enc(sample_x)
#print(sample_x.shape)
#sample_x = sample_dec(sample_x)
#print(sample_x.shape)
sample_x, weights = sample_tf()
print(sample_x.shape)


torch.Size([13, 29])


# TRAIN

In [9]:
import torch 
import torch.nn as nn
import random
import numpy as np

EPOCHS = 200

transformer = Transformer()
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0002)
loss = torch.nn.BCELoss()

real_sample = torch.Tensor([
        [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
])
for _ in range(1000):
    sample, __ = transformer()        
    # target = torch.ones(sample.shape[0], sample.shape[1])
    error = loss(sample, real_sample)
    if _ % 10 == 0: print(error, sample)
    error.backward()
    optimizer.step()
    if(error > 8): break
print(sample)


tensor(0.1623, grad_fn=<BinaryCrossEntropyBackward>) tensor([[0.0458, 0.0297, 0.0645, 0.0065, 0.0237, 0.0166, 0.0172, 0.0396, 0.0163,
         0.0108, 0.0347, 0.0145, 0.1121, 0.0391, 0.0214, 0.1197, 0.0355, 0.0254,
         0.0427, 0.0232, 0.0339, 0.0335, 0.0547, 0.0071, 0.0352, 0.0327, 0.0146,
         0.0285, 0.0208],
        [0.0438, 0.0275, 0.0617, 0.0219, 0.0498, 0.0380, 0.0243, 0.0398, 0.0239,
         0.0386, 0.0202, 0.0120, 0.0439, 0.0247, 0.0242, 0.0821, 0.0359, 0.0226,
         0.0525, 0.0494, 0.0451, 0.0157, 0.0756, 0.0106, 0.0174, 0.0363, 0.0232,
         0.0164, 0.0229],
        [0.0354, 0.0459, 0.0304, 0.0529, 0.0376, 0.0258, 0.0271, 0.0185, 0.0492,
         0.0327, 0.0528, 0.0063, 0.0360, 0.0091, 0.0304, 0.0433, 0.0152, 0.0333,
         0.0446, 0.0463, 0.0303, 0.0256, 0.0681, 0.0194, 0.0475, 0.0565, 0.0271,
         0.0330, 0.0197],
        [0.0295, 0.0687, 0.0269, 0.0250, 0.0322, 0.0169, 0.0249, 0.0382, 0.0306,
         0.0364, 0.0398, 0.0118, 0.0300, 0.0404, 0.0288, 0.

tensor(0.0923, grad_fn=<BinaryCrossEntropyBackward>) tensor([[0.2667, 0.0010, 0.0004, 0.0007, 0.1040, 0.0006, 0.0005, 0.2786, 0.0007,
         0.0005, 0.0011, 0.0053, 0.0005, 0.0005, 0.0032, 0.0021, 0.0008, 0.0004,
         0.0005, 0.3230, 0.0008, 0.0023, 0.0016, 0.0007, 0.0005, 0.0005, 0.0010,
         0.0006, 0.0007],
        [0.1221, 0.0017, 0.0009, 0.0010, 0.1843, 0.0021, 0.0011, 0.3974, 0.0011,
         0.0007, 0.0008, 0.0123, 0.0005, 0.0006, 0.0023, 0.0011, 0.0010, 0.0006,
         0.0006, 0.2547, 0.0017, 0.0027, 0.0011, 0.0009, 0.0006, 0.0019, 0.0019,
         0.0018, 0.0006],
        [0.1306, 0.0017, 0.0006, 0.0006, 0.2587, 0.0019, 0.0009, 0.2510, 0.0009,
         0.0013, 0.0008, 0.0072, 0.0008, 0.0011, 0.0015, 0.0013, 0.0020, 0.0008,
         0.0021, 0.3222, 0.0020, 0.0018, 0.0019, 0.0011, 0.0004, 0.0010, 0.0016,
         0.0010, 0.0011],
        [0.2368, 0.0011, 0.0012, 0.0007, 0.2292, 0.0006, 0.0021, 0.1540, 0.0007,
         0.0012, 0.0006, 0.0055, 0.0004, 0.0009, 0.0024, 0.

tensor(0.0774, grad_fn=<BinaryCrossEntropyBackward>) tensor([[0.0279, 0.0004, 0.0002, 0.0001, 0.0247, 0.0002, 0.0002, 0.0095, 0.0003,
         0.0002, 0.0002, 0.1123, 0.0001, 0.0002, 0.0002, 0.0004, 0.0002, 0.0002,
         0.0002, 0.8206, 0.0002, 0.0002, 0.0003, 0.0001, 0.0001, 0.0001, 0.0001,
         0.0001, 0.0002],
        [0.0083, 0.0003, 0.0001, 0.0001, 0.0210, 0.0001, 0.0001, 0.0087, 0.0001,
         0.0002, 0.0001, 0.1832, 0.0001, 0.0001, 0.0001, 0.0002, 0.0004, 0.0001,
         0.0002, 0.7749, 0.0002, 0.0002, 0.0002, 0.0001, 0.0001, 0.0003, 0.0002,
         0.0001, 0.0002],
        [0.0506, 0.0008, 0.0004, 0.0003, 0.0523, 0.0003, 0.0006, 0.0137, 0.0007,
         0.0003, 0.0006, 0.1117, 0.0003, 0.0003, 0.0005, 0.0003, 0.0006, 0.0002,
         0.0007, 0.7612, 0.0003, 0.0004, 0.0004, 0.0003, 0.0004, 0.0003, 0.0006,
         0.0007, 0.0003],
        [0.0164, 0.0004, 0.0001, 0.0001, 0.0171, 0.0003, 0.0001, 0.0059, 0.0001,
         0.0001, 0.0002, 0.1187, 0.0001, 0.0002, 0.0003, 0.

tensor(0.1137, grad_fn=<BinaryCrossEntropyBackward>) tensor([[2.0813e-04, 2.6811e-05, 9.8765e-06, 6.8617e-06, 5.3598e-04, 7.8464e-06,
         1.8330e-05, 1.8987e-03, 7.5485e-06, 1.0931e-05, 2.6748e-05, 5.2302e-01,
         2.0994e-05, 1.6651e-05, 1.1009e-05, 1.5005e-05, 3.4297e-05, 9.0911e-06,
         1.3892e-05, 4.7397e-01, 1.1904e-05, 8.4866e-06, 2.0223e-05, 9.9141e-06,
         2.0379e-05, 9.6738e-06, 1.8600e-05, 1.5369e-05, 2.1521e-05],
        [9.0121e-04, 2.4877e-05, 9.9112e-06, 6.3887e-06, 3.0030e-04, 1.1330e-05,
         2.3810e-05, 7.1602e-04, 2.4188e-05, 1.6396e-05, 1.1730e-05, 5.2531e-01,
         1.8639e-05, 2.3811e-05, 1.6399e-05, 2.3828e-05, 1.9584e-05, 5.9798e-06,
         2.4996e-05, 4.7235e-01, 4.8399e-05, 1.1416e-05, 1.6903e-05, 7.6824e-06,
         5.4634e-06, 1.1094e-05, 2.2555e-05, 1.4871e-05, 2.1470e-05],
        [4.3988e-04, 2.0615e-05, 6.4027e-06, 5.2274e-06, 4.4107e-04, 7.2721e-06,
         8.5958e-06, 6.0835e-04, 5.7839e-06, 5.7265e-06, 1.2369e-05, 5.9899e-0

tensor(0.0998, grad_fn=<BinaryCrossEntropyBackward>) tensor([[8.0591e-04, 3.5764e-05, 6.1882e-06, 2.5883e-06, 9.2574e-04, 9.2861e-06,
         5.7445e-06, 2.0154e-03, 1.5215e-05, 5.9099e-06, 7.7091e-06, 3.9559e-01,
         9.1744e-06, 6.4281e-06, 8.8118e-06, 1.9737e-05, 2.3332e-05, 3.1312e-06,
         1.3778e-05, 6.0039e-01, 1.1651e-05, 4.9529e-06, 1.0500e-05, 9.3253e-06,
         9.2690e-06, 5.6933e-06, 1.9834e-05, 1.4606e-05, 1.4186e-05],
        [7.2390e-04, 7.9865e-06, 6.9582e-06, 3.3067e-06, 6.4597e-04, 2.5622e-06,
         3.1968e-06, 7.1446e-04, 3.9689e-06, 1.8749e-06, 4.7680e-06, 2.8428e-01,
         3.2936e-06, 5.0294e-06, 7.7588e-06, 1.1753e-05, 2.3716e-06, 3.1151e-06,
         4.6029e-06, 7.1352e-01, 8.3217e-06, 2.2601e-06, 6.7185e-06, 3.3696e-06,
         2.1046e-06, 4.5398e-06, 6.6094e-06, 2.8473e-06, 4.8174e-06],
        [1.0851e-03, 1.9808e-05, 3.8341e-06, 9.6210e-06, 9.1870e-04, 1.0711e-05,
         7.5953e-06, 1.6063e-03, 1.3621e-05, 6.1888e-06, 1.1568e-05, 3.4835e-0

...

tensor(0.1937, grad_fn=<BinaryCrossEntropyBackward>) tensor([[4.9187e-02, 1.4883e-12, 2.3995e-12, 8.8921e-13, 1.7853e-04, 8.2670e-13,
         8.2816e-12, 2.3014e-06, 4.1932e-13, 5.9953e-13, 5.8351e-13, 9.8618e-08,
         4.9694e-12, 5.2168e-13, 3.1191e-13, 3.1744e-12, 6.5732e-13, 1.0134e-12,
         2.9232e-13, 9.5063e-01, 1.9762e-13, 5.5095e-13, 1.7918e-11, 4.7531e-13,
         2.1988e-12, 4.1496e-13, 2.3531e-13, 7.4917e-13, 2.8094e-13],
        [3.5949e-01, 1.2234e-10, 7.2603e-11, 6.2775e-11, 1.5288e-04, 7.0369e-11,
         2.9022e-10, 2.5687e-06, 7.6370e-11, 3.2867e-11, 5.7065e-11, 5.0356e-08,
         2.3201e-10, 7.7199e-11, 6.1672e-12, 7.4166e-11, 5.0711e-11, 5.5048e-11,
         3.7513e-11, 6.4036e-01, 2.6897e-11, 2.4033e-11, 5.4661e-10, 3.8252e-11,
         1.3070e-10, 6.1566e-11, 2.9047e-12, 5.7441e-11, 1.7803e-11],
        [8.7340e-02, 4.1245e-10, 1.5723e-10, 1.3415e-10, 1.3246e-04, 1.7001e-10,
         1.0617e-09, 8.5107e-07, 8.7692e-11, 1.0780e-10, 1.6377e-10, 3.2994e-0

tensor(0.1926, grad_fn=<BinaryCrossEntropyBackward>) tensor([[1.5083e-01, 1.3873e-10, 1.5595e-11, 4.6588e-11, 3.6143e-04, 2.0216e-11,
         9.5711e-11, 6.2985e-07, 2.7818e-11, 2.4609e-11, 3.1297e-11, 1.1148e-07,
         1.0330e-10, 2.1165e-11, 2.3321e-12, 7.9100e-11, 1.7590e-11, 2.0129e-11,
         8.4939e-12, 8.4880e-01, 6.7847e-12, 1.1886e-11, 3.4249e-10, 6.1391e-12,
         3.8267e-11, 2.1496e-11, 1.5750e-12, 1.5028e-11, 1.5669e-11],
        [1.1262e-01, 2.5188e-13, 2.0225e-13, 4.5462e-13, 2.0957e-04, 2.2584e-13,
         3.5806e-13, 3.9663e-07, 1.9933e-13, 1.3211e-13, 8.9574e-14, 1.2041e-07,
         3.4050e-13, 2.6776e-13, 1.9072e-13, 2.6064e-13, 3.1204e-13, 1.7428e-13,
         1.9578e-13, 8.8717e-01, 1.5413e-13, 7.8470e-14, 7.2120e-12, 1.0734e-13,
         3.1913e-13, 1.4980e-13, 4.9515e-14, 3.0552e-13, 9.2608e-14],
        [7.9015e-02, 9.2949e-11, 1.7135e-11, 1.5463e-11, 4.2920e-04, 1.9481e-11,
         9.0379e-11, 6.6156e-07, 2.4274e-11, 3.3145e-11, 2.7324e-11, 1.6649e-0

tensor(0.1829, grad_fn=<BinaryCrossEntropyBackward>) tensor([[5.1138e-01, 9.7576e-11, 4.5048e-11, 3.3034e-11, 2.5549e-03, 3.0007e-11,
         4.3015e-10, 1.5891e-06, 3.2838e-11, 2.9064e-11, 3.7289e-11, 2.9584e-07,
         2.3323e-10, 1.8523e-11, 5.8828e-12, 3.3530e-10, 7.2901e-12, 1.1832e-11,
         9.7596e-12, 4.8606e-01, 2.2723e-12, 7.8529e-12, 2.5122e-09, 8.2594e-12,
         9.1559e-11, 9.6799e-12, 1.2127e-12, 2.2909e-11, 3.6638e-12],
        [3.2031e-01, 5.3548e-11, 9.6265e-12, 2.3352e-11, 7.1205e-04, 9.7280e-12,
         1.5918e-11, 8.9362e-07, 1.8330e-11, 1.8680e-11, 1.5284e-11, 3.9230e-07,
         8.3283e-11, 7.2411e-12, 1.1652e-12, 4.9565e-11, 2.2280e-12, 5.0356e-12,
         5.7722e-12, 6.7897e-01, 1.6313e-12, 2.2186e-12, 1.3255e-10, 6.4446e-12,
         5.4217e-11, 5.2735e-12, 5.2872e-13, 9.1996e-12, 8.0073e-12],
        [4.1480e-01, 1.5201e-10, 9.1500e-11, 4.1185e-11, 5.6023e-03, 3.2941e-11,
         3.0021e-10, 9.3146e-07, 2.7729e-11, 4.3965e-11, 3.8694e-11, 1.2771e-0

tensor(0.2118, grad_fn=<BinaryCrossEntropyBackward>) tensor([[7.2167e-01, 1.9618e-07, 7.6869e-08, 3.9481e-08, 2.5815e-02, 5.7579e-08,
         9.2127e-08, 2.2195e-06, 2.3858e-08, 4.3036e-08, 3.4392e-08, 1.2504e-05,
         2.0309e-07, 3.1492e-08, 5.6709e-09, 3.8661e-08, 1.3472e-08, 1.4783e-08,
         1.5057e-08, 2.5250e-01, 2.4383e-09, 5.9821e-09, 8.7189e-07, 7.8199e-09,
         1.1961e-07, 1.0056e-08, 2.0112e-09, 1.5294e-08, 8.5694e-09],
        [8.1798e-01, 4.7120e-10, 2.8204e-10, 2.0582e-10, 4.8965e-03, 1.2462e-10,
         9.2844e-10, 6.2531e-07, 1.1277e-10, 1.9691e-10, 1.0970e-10, 5.4993e-07,
         1.5913e-09, 6.9723e-11, 5.0407e-12, 1.6688e-09, 2.7177e-11, 6.0487e-11,
         2.3019e-11, 1.7713e-01, 2.1851e-11, 3.5623e-11, 2.4782e-09, 7.6475e-12,
         2.8748e-10, 1.2682e-10, 4.0409e-12, 1.0485e-10, 8.1624e-11],
        [6.4155e-01, 2.0112e-10, 6.5305e-10, 2.3168e-10, 2.4592e-02, 2.7073e-10,
         1.4746e-09, 2.8459e-06, 1.6264e-10, 3.1519e-10, 2.2958e-10, 3.1164e-0

tensor(0.2060, grad_fn=<BinaryCrossEntropyBackward>) tensor([[4.9793e-01, 9.0754e-10, 2.7323e-10, 7.9852e-11, 2.6481e-02, 9.9807e-11,
         4.2532e-09, 2.3809e-06, 6.7226e-11, 8.5219e-10, 4.5518e-10, 7.0932e-06,
         1.4589e-09, 4.7215e-11, 1.2471e-11, 3.1714e-09, 3.3628e-11, 4.7730e-11,
         4.5728e-11, 4.7558e-01, 2.7698e-11, 2.8785e-11, 2.7677e-08, 4.7095e-11,
         3.4164e-10, 7.8403e-11, 5.5121e-12, 1.1989e-10, 6.3105e-11],
        [8.6886e-01, 3.9761e-11, 6.5891e-12, 8.8430e-12, 2.4109e-02, 6.0795e-12,
         9.9909e-11, 4.2389e-07, 6.1049e-12, 1.5399e-11, 2.6844e-11, 1.3512e-06,
         4.3591e-11, 8.3458e-12, 1.3897e-11, 6.8198e-11, 6.5850e-12, 5.1682e-12,
         4.8530e-12, 1.0703e-01, 5.3549e-12, 2.2012e-12, 2.3516e-10, 2.6636e-12,
         1.1543e-11, 4.4282e-12, 1.5040e-12, 7.7830e-12, 4.6708e-12],
        [5.9195e-01, 3.5956e-09, 3.4843e-09, 5.9307e-10, 4.2734e-02, 1.0051e-09,
         4.2317e-09, 1.2026e-06, 6.7632e-10, 2.4527e-09, 1.7908e-09, 1.3982e-0

In [ ]:
print(torch.argmax(real_sample, dim=1))
sample, _ = transformer()
print(torch.argmax(sample, dim=1))

In [ ]:
print(__)